### 아이디어
- 기존에 풀었던 스위핑 세그 문제처럼 한 축을 잡고 스위핑하면서 푼다고 했을 때, 곤란해지는 지점은 y축이나 x축 위에 여러개의 광산이 있을때이다. 
- 광산의 개수가 3000개로, 구분 가능한 x, y좌표는 6000개만 나온다. \
만약 가능한 x, y좌표에 대해 모든 쌍을 구한다면 $6000^2$ 개 정도가 나오는데, 만약 쿼리를 구하는 것이 $\log N$ 정도라면 충분히 구할 수 있을 것이다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from dataclasses import dataclass
@dataclass
class Node :
  ls: int = 0
  rs: int = 0
  ms: int = 0
  s: int = 0

class ST : 
  def __init__(self, size): #default는 Node()이다.
    self._size = _size = 1 << (size - 1).bit_length()
    self.seg = [Node() for _ in range(2 * _size)]

  def __getitem__(self, i):
    return self.seg[i + self._size]

  def update(self, i, v) :
    i += self._size
    self.seg[i].ms += v
    self.seg[i].ls = self.seg[i].rs = self.seg[i].s = self.seg[i].ms
    while i > 1 :
      i >>= 1
      self.seg[i] = self.merge(self.seg[i*2], self.seg[i*2+1])

  def merge(self, l: Node, r: Node):
    ret = Node()
    ret.s = l.s + r.s
    ret.ls = max(l.ls, l.s + r.ls)
    ret.rs = max(r.rs, r.s + l.rs)
    ret.ms = max(ret.s, l.ms, r.ms, l.rs + r.ls)
    return ret

  def query(self, s, e): #[s, e]
    s += self._size - 1
    e += self._size

    ret = Node()
    while s < e :
      if s & 1 :
        ret = self.merge(ret, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        ret = self.merge(self.seg[e], ret)
      s >>= 1
      e >>= 1
    return ret

def sol() :
  N = int(input())
  X = set()
  Y = set()
  T = []
  for _ in range(N) :
    x, y, w = map(int, input().split())
    T.append([x, y, w])
    X.add(x)
    Y.add(y)
  
  X = {x: i for i, x in enumerate(sorted(X))}
  Y = {y: i for i, y in enumerate(sorted(Y))}
  T = [[X[x], Y[y], w] for x, y, w in T] #compression
  T.sort(key=lambda x: (x[1], x[0]))

  ans = 0
  for i in range(N) :
    if i and T[i][1] == T[i-1][1] : continue #같은 y좌표는 한번만
    st = ST(len(X) + len(Y))
    for j in range(i, N) :
      st.update(T[j][0], T[j][2])
      if j == N - 1 or T[j][1] != T[j+1][1] :
        ans = max(ans, st.seg[1].ms)  
  print(ans)

sol()

### 풀이
- 가능한 x, y 좌표 쌍에 대해 모두 시도해보는 아이디어는 맞았다.
- 16993(연속합과 쿼리)에서 사용된 최대 연속누적합 세그트리의 아이디어를 사용하여 구역 $p_1, p_2$ 에 대한 개발 이익을 $\Omicron(\log N)$ 에 구할 수 있다.
- $y$ 기준으로 좌표를 정렬한 뒤, $x$ 기준으로 스위핑하면서 구간 $[p_1, p_2]$ 에 대한 개발 이익을 구한다. 그 과정에서 쿼리의 최댓값이 정답이다.